In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import uuid

def generate_uuid():
    return str(uuid.uuid4())
uuid_udf = udf(generate_uuid, StringType())

df_title_AKAS = spark.read.table("imdb_dev.silver.TitleAKAS") 
df_type_AKAS = spark.read.table("imdb_dev.silver.TypeAKAS") 
df_type_of_alternative = spark.read.table("imdb_dev.silver.TypeOfAlternative") 
df_region = spark.read.table("imdb_dev.silver.Region") 

df_title_AKAS = (
    df_title_AKAS.join(df_region, on="regionId", how="left")
    .drop("regionId", "title")
)
df_type_of_alternative = (
    df_type_of_alternative.join(df_type_AKAS, on="typeAKASId", how="left")
    .withColumnRenamed("typeAKAS", "AKAStype")
    .drop("typeAKASId")
)
df_AKAS_bridge = (
    df_title_AKAS.join(df_type_of_alternative, on="titleAKASId", how="left")
    .drop("titleAKASId")
)

In [0]:
df_AKAS_dim = (
    df_AKAS_bridge.drop("titleId")
    .distinct()
    .withColumn("AKASId", uuid_udf())
)
df_AKAS_dim = df_AKAS_dim.select("AKASId", "AKAStype", "region")
df_AKAS_dim.write.format("delta").mode("overwrite").insertInto("imdb_dev.gold.AKASDim")

In [0]:
df_AKAS_dim = spark.read.table("imdb_dev.gold.AKASDim") 

df_AKAS_bridge = (
    df_AKAS_bridge
    .join(df_AKAS_dim, on=["region", "AKAStype"], how='left')
    .drop("region", "AKAStype")
)
df_AKAS_bridge = df_AKAS_bridge.select("AKASId", "titleId")
df_AKAS_bridge.write.format("delta").mode("overwrite").insertInto("imdb_dev.gold.AKASBridge")